# Initialization

In [17]:
import os
import numpy as np
import pickle
import json
import pandas as pd
from grabbit import Layout
from mne import read_epochs, grand_average, write_evokeds, read_evokeds
from mne import pick_types, combine_evoked, set_log_level
from mne.viz import plot_compare_evokeds
from msit import CH_NAMES, select_subjects, add_events, drop_bad_trials
import matplotlib.pyplot as plt
from mne import read_evokeds

set_log_level('critical')

# load subject names to process
layout = Layout('../data', '../data/grabbit_config.json')
subjects = select_subjects(layout, 'eeg', exclude='eeg')

# load eeg configurations
with open('../config.json') as f:    
    config = json.load(f)

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv', 
                       na_values='n/a', sep='\t') 
behavior = behavior[behavior.modality == 'eeg']

pipeline_root = '../data/derivatives/eeg_sensor_erps'

# Make Directory Structure

In [2]:
# Make directory structure
if not os.path.exists(pipeline_root):
    os.makedirs(pipeline_root)
for subject in subjects + ['group']:
    if not os.path.exists('%s/%s' % (pipeline_root, subject)):
        os.makedirs('%s/%s' % (pipeline_root, subject))

# Make Evoked Data

In [18]:
for epo_type, epo_times in zip(config['epoch_types'], 
                               config['epoch_times']):
    print(epo_type)
    
    for event_type in config['event_types']:
        print(event_type)
        mne_evokeds = {}
        evokeds = {}
        naves = {}
        ses = {}
        for event in config[event_type]:
            mne_evokeds[event] = []
            evokeds[event] = []
            ses[event] = []
            naves[event] = []
        bads = []
    
        for subject in subjects:
            print(subject)

            # load subject behavior and epochs
            sub_behavior = behavior[behavior.participant_id == subject]
            epo_file = layout.get(subject=subject, 
                                  derivative='eeg_preprocessing', 
                                  extensions='%s_cleaned-epo.fif' % epo_type)[0]
            epochs = read_epochs(epo_file.filename, verbose=False)

            # remove filter buffer 
            epochs.crop(epo_times[0], epo_times[1])
            # interpolate the bad channels
            bads.append(epochs.info['bads'])
            epochs.interpolate_bads(reset_bads=True)
            # drop bad trials from epochs and behavior
            sub_behavior, epochs = drop_bad_trials(subject, sub_behavior, 
                                                   epochs, layout, epo_type)

            # add event labels
            epochs = add_events(epochs, sub_behavior, event_type,
                                config[event_type])

            # compute averages and standard errors 
            evos = [epochs[event].average() for event in config[event_type]]
            evos_se = [epochs[event].standard_error() 
                       for event in config[event_type]]
            
            # write subject evoked objects to file
            write_evokeds('%s/%s/%s_%s_%s-ave.fif' % (pipeline_root,
                                                      subject, subject,
                                                      event_type, epo_type), 
                          evos)
            # accumulate evoked objects for group average topomap
            for i, event in enumerate(config[event_type]):
                mne_evokeds[event].append(evos[i])
            
            # extract info for group erp object
            for i, event in enumerate(config[event_type]):
                evokeds[event].append(evos[i].data)
                ses[event].append(evos_se[i].data)
                naves[event].append(evos[i].nave)
            
        # save out group erp information
        for event in config[event_type]:
            evokeds[event] = np.array(evokeds[event])
            ses[event] = np.array(ses[event])
            naves[event] = np.array(naves[event])
            
        f = '%s/group/group_%s_%s_erp_info.npz' % (pipeline_root, 
                                                   event_type, epo_type)
        np.savez_compressed(f, evoked=evokeds, standard_error=ses,
                            num_ave=naves, chs=evos[0].ch_names, 
                            times=evos[0].times)
            

        # calculate group average evoked object and save 
        evos = [combine_evoked(mne_evokeds[event], weights='nave')
                for event in config[event_type]]
        write_evokeds('%s/group/group_%s_%s-ave.fif' % (pipeline_root, 
                                                        event_type,
                                                        epo_type), evos)


print('Done!')

stimulus
condition
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc019
sub-hc020
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc037
sub-hc042
sub-hc044
sub-hc045
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011


KeyboardInterrupt: 

# Plot Topomaps 

In [43]:
from mne import read_evokeds
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

def plot_topomap(subject, epo_type, event_type, time, col_limit):
    plt.close('all')
    f = '%s/%s/%s_%s_%s-ave.fif' % (pipeline_root, subject, subject,
                                          event_type, epo_type)
    evokeds = read_evokeds(f)
    
    f, axs = plt.subplots(1, len(evokeds), figsize=(6 * len(evokeds), 6))
    
    for i, evo in enumerate(evokeds):
        evo.plot_topomap(times=time, axes=axs[i], colorbar=False, 
                         show=False, vmin=-col_limit, vmax=col_limit)
        axs[i].set_xlabel(config[event_type][i])
    plt.show()

interact(plot_topomap, subject=['group'] + subjects, 
         epo_type=['response', 'stimulus'], event_type=config['event_types'],
         time=np.arange(-1, 1.75, .01), col_limit=np.arange(.5, 5, .5))

<function __main__.plot_topomap>

In [11]:
from ipywidgets import interact

def plot_erp(subject, ch):

    fig, axs = plt.subplots(1, 2, figsize=(24, 8))

    for i, epo_type in enumerate(config['epoch_types']):

        ax = axs[i]

        evokeds = read_evokeds('%s/%s/%s_%s-ave.fif' % (pipeline_root,
                                                        subject,
                                                        subject,
                                                        epo_type),
                              verbose=False)
        pick = evokeds[0].ch_names.index(ch)
        evokeds = dict((config['conditions'][i], evokeds[i]) 
                       for i in range(len(evokeds)))
        plot_compare_evokeds(evokeds, picks=pick, axes=ax, show=False)
        ax.set_title('%s-locked' % epo_type)
        ax.set_ylim((-6, 6))
        ax.axvline(0, linestyle='--', color='k')

    plt.suptitle('%s %s ERP' % (subject, ch))
    plt.show()

interact(plot_erp, subject=subjects + ['group'], ch=CH_NAMES)

<function __main__.plot_erp>

In [7]:
plot_compare_evokeds?